In [2]:
import boto3
import csv
import json
import random

In [3]:
# path = 'C:/Users/jordi/Documents/claus/'
path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [24]:
language = "spanish"

In [39]:
# The code below is to pick random document codes from the list of documents with incentives

path = "../out/"
file_name = f"incentives_{language}.csv"
file = path + file_name

with open(file, newline='') as f:
    reader = csv.reader(f)
    next(reader)
    file_code = [row[0] for row in reader]

codes = {}
for item in random.sample(file_code, 50):
    codes[item] = 0

In [40]:
# To pick the sentences refering to incentives from the selected documents

in_prefix = f"{language}_documents/HSSC/updated_sentences/"  

i = 0
documents_with_incentives = 0
results_list = [["Sentence_name", "Sentence text", "Label"]]

incentives = 0
for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):
    name = obj.key.split("/")[3].split("_")[0]
#     print(name)
    if name in codes :
        sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
        for sentence_id in sentences[name]["sentences"]:
            incentive = sentences[name]["sentences"][sentence_id]["label"]
            if len(incentive) > 1 and incentive[0] == "Incentive":
                incentives += 1
                results_list.append([])
                results_list[incentives].append(name)
                results_list[incentives].append(sentences[name]["sentences"][sentence_id]["text"])
                results_list[incentives].append(incentive[1])
                
with open(f"../out/incentive_sentences_{language}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(results_list)           

In [25]:
# The code below is to retrieve de document codes of the sentences which refer to incentives and 
# that has been already picked for QA

path = "../out/"
file_name = f"incentive_sentences_{language}.csv"
file = path + file_name

with open(file, newline='') as f:
    reader = csv.reader(f)
    next(reader)
    file_code = [row[0].split("_")[0] for row in reader]

codes = {}
for item in file_code:
    codes[item] = 0

In [27]:
# To pick up the sentences refering to non-incentives from the selected documents

in_prefix = f"{language}_documents/HSSC/updated_sentences/"  

i = 0
documents_with_incentives = 0
results_list = [["Sentence_name", "Sentence text", "Label"]]


for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):
    name = obj.key.split("/")[3].split("_")[0]
#     print(name)
    if name in codes :
        sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
        n_incentives = incentive = sentences[name]["metadata"]["number_of_incentives"]
        print(n_incentives)
        non_incentives = [[]]
        incentives = 0
        for sentence_id in sentences[name]["sentences"]:
            incentive = sentences[name]["sentences"][sentence_id]["label"]
            if len(incentive) == 1 and incentive[0] == "not_Incentive":
                non_incentives.append([])
                incentives += 1
                non_incentives[incentives].append(name)
                non_incentives[incentives].append(sentences[name]["sentences"][sentence_id]["text"])
                non_incentives[incentives].append(incentive[0])
        
        for item in random.sample(non_incentives, n_incentives):
            results_list.append(item)
                
                
with open(f"../out/non_incentive_sentences_{language}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(results_list)          

2
1
4
7
8
1
20
5
1
1
1
7
53
2
65
1
1
2
6
1
1
2
2
2
6
9
19
4
31
1
15
1
1
1
26
12
1
20
16
1
1
1
5
9
1
4
6
1
3
16


In [26]:
codes

{'06bfba1dca46ddf3ae89d3fb754acf0335778a93': 0,
 '0750714d6ada19f6979086d5797adb3c54ee88a8': 0,
 '09580605e54488941cd2dea454923dc84a1e1472': 0,
 '0969fa6f52544e666f9064fe2d8c70933bffa7d8': 0,
 '0df7a216166cbb4e74ed43913b7dbfee480934f8': 0,
 '177852c775414040f2a37b098f1d9b88a7639717': 0,
 '1bec287d4fa6cfb475f4835d2c1002e1ad1be2be': 0,
 '1f55386e6e10651f946717edfceee4afa98b1205': 0,
 '25175eb0b599c1f85f0b31b317710bba42cc29e8': 0,
 '316a4d4e2946da5fd5755884c0f852b7b49e65d4': 0,
 '3b38277d3a47b85711bfe8486fb9ef9b70a09ceb': 0,
 '3c2ee0806e559acdfae40764979a7d0b71560a3d': 0,
 '3f3dc1abd094d7918da396454293d97303cae340': 0,
 '4197ba51f91d2828ebee052e42dab4f2ae634060': 0,
 '4873e2b1bc6c19ff2a2ff9fdf82881950b3d2c0f': 0,
 '4acd3d9cd7ac4dd5b987839e41cae856563273ce': 0,
 '4c99695b6ed85c291bb6302e40e1f4adceeecee4': 0,
 '4ff99954d90cd3d6a0f76601c7c93e4d60dfc800': 0,
 '522fea89babeeb86ee67e5a430fff1a56935bd83': 0,
 '5235ba6a92cc92aa9c4d164771874a53aaa30ad2': 0,
 '5afb2c6017d71bae2e93dd12adc9d420080ffa

In [5]:
codes

{'03eb3072facaf6545ad38b03eafc5f3c2321a32f': 0,
 '057a30c54f8c1dad4e14f7d263abdc7263459db1': 0,
 '0f385d3089bdedad513a55269882a6e5e77b3690': 0,
 '0f81d812c40dff5cb946ecb42b3dda6182b8eaf1': 0,
 '10fea2073ed580168ff6bc383f37ed968f4f95b7': 0,
 '186fd3642efd58a3c66d3fbcf4c55988fcd5ece1': 0,
 '18ff9e535bf5c9697824901f5bc2f6854700da31': 0,
 '1d4756b6080454bc595852ede312f2b213401fe2': 0,
 '1e74676fb5620d05845f8e2821c004a6d2c5402a': 0,
 '2381fedaffaa1f5bf783069e661f15b66c20171b': 0,
 '28f5a271eb73d86b199c23db4c6f7dec836b8849': 0,
 '2af22347a13b7d47d7f45354d7a4b8275d7869a6': 0,
 '30b674bcdc858de469e1aa6bf701087a13134f85': 0,
 '33e1959b5491d1945d67889b3c9da39e2ac4d759': 0,
 '3dfaa4bd5f7d4755c2198761d76c4b6eaae2d48f': 0,
 '4382649c3f24724e3c1a99dd15dba9840a5e4155': 0,
 '459caa4c2fda39622ef5dbdca09529b3815336d4': 0,
 '4ab4646fb95565ac8a4774c4c3fdf7b16f32dc9c': 0,
 '52daef3bf3cff993d034407dae5bef67cbe6231a': 0,
 '5b37ed22b647559cbed9dc07ada0d29816c6acb8': 0,
 '5ba1969bc96226edcc309862259bb2e5d30452